# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
#print(geoapify_key)  

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,28.99,74,75,6.69,CC,1714970113
1,1,butterworth,5.3991,100.3638,32.03,66,20,3.60,MY,1714970113
2,2,kapa'a,22.0752,-159.3190,26.99,82,20,7.20,US,1714970114
3,3,vilyuchinsk,52.9306,158.4028,7.15,72,51,3.50,RU,1714970114
4,4,tlaltenango de sanchez roman,21.7833,-103.3167,20.16,25,91,1.64,MX,1714970114
5,5,adamstown,-25.0660,-130.1015,21.57,71,67,9.45,PN,1714970114
6,6,sisimiut,66.9395,-53.6735,-9.44,89,63,3.54,GL,1714969880
7,7,thompson,55.7435,-97.8558,4.09,75,20,1.54,CA,1714970115
8,8,port-aux-francais,-49.3500,70.2167,5.84,79,88,12.57,TF,1714970115
9,9,iqaluit,63.7506,-68.5145,-4.15,86,100,5.66,CA,1714970115


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    size=city_data_df['Humidity'],
    hover_cols=['City', 'Humidity'],
    color='City',  # Use a different column for city colors
    cmap='Category20',  # Specify a colormap for the colors
    alpha=0.6,
    tiles='OSM',
    frame_width=700,
    frame_height=500, 
    legend=True  # Display a legend on the side 
    
)


# Display the map
hvplot.show(humidity_map)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Define your ideal weather conditions 
ideal_conditions = {
    "Max Temp": (21, 27),  # Temperature in Celsius
    "Humidity": (0, 50),   # Humidity in percentage
    "Wind Speed": (0, 10)  # Wind speed in m/s
}


# Filter the DataFrame based on the ideal conditions
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_conditions['Max Temp'][0]) &
    (city_data_df['Max Temp'] <= ideal_conditions['Max Temp'][1]) &
    (city_data_df['Humidity'] >= ideal_conditions['Humidity'][0]) &
    (city_data_df['Humidity'] <= ideal_conditions['Humidity'][1]) &
    (city_data_df['Wind Speed'] >= ideal_conditions['Wind Speed'][0]) &
    (city_data_df['Wind Speed'] <= ideal_conditions['Wind Speed'][1])
]

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,i-n-salah,27.1935,2.4607,26.87,12,0,8.41,DZ,1714970118
48,48,port alfred,-33.5906,26.8910,22.32,40,16,5.27,ZA,1714970121
53,53,emiliano zapata,18.8333,-99.1833,25.00,47,100,1.58,MX,1714969946
92,92,margaret river,-33.9500,115.0667,21.06,50,31,3.63,AU,1714970128
109,109,tsengel,49.4783,100.8894,21.58,21,48,2.80,MN,1714970131


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,31,i-n-salah,27.1935,2.4607,26.87,12,0,8.41,DZ,1714970118,
48,48,port alfred,-33.5906,26.8910,22.32,40,16,5.27,ZA,1714970121,
53,53,emiliano zapata,18.8333,-99.1833,25.00,47,100,1.58,MX,1714969946,
92,92,margaret river,-33.9500,115.0667,21.06,50,31,3.63,AU,1714970128,
109,109,tsengel,49.4783,100.8894,21.58,21,48,2.80,MN,1714970131,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE


# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},{radius}',
        'limit': 1,
        'apiKey': geoapify_key
    }
    
    # Set base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df

Starting hotel search...
i-n-salah - nearest hotel: تيدكلة
port alfred - nearest hotel: Ferryman
emiliano zapata - nearest hotel: Ex-Hacienda de Cortes.
margaret river - nearest hotel: Quality Inn Margaret River
tsengel - nearest hotel: No hotel found
dingtao - nearest hotel: No hotel found
inda silase - nearest hotel: ቃሌማ ሆቴል
murzuq - nearest hotel: فندق باريس - مرزق
bahia de kino - nearest hotel: Casa Blanca Inn
saint-pierre - nearest hotel: La Villa Delisle
honjo - nearest hotel: 埼玉グランドホテル本庄
uliastay - nearest hotel: Zagasan Nud Hotel
fuerte olimpo - nearest hotel: No hotel found
ust'-kut - nearest hotel: Домашний очаг
carnarvon - nearest hotel: No hotel found
zheleznogorsk-ilimskiy - nearest hotel: Магнетит
medina - nearest hotel: أجنحة فيردان للوحدات السكنية
jiangyou - nearest hotel: No hotel found
jinchang - nearest hotel: No hotel found
hafar al-batin - nearest hotel: فندق الفرحان
aoulef - nearest hotel: دار الضياف
kutum - nearest hotel: No hotel found
east london - nearest hote

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,31,i-n-salah,27.1935,2.4607,26.87,12,0,8.41,DZ,1714970118,تيدكلة
48,48,port alfred,-33.5906,26.8910,22.32,40,16,5.27,ZA,1714970121,Ferryman
53,53,emiliano zapata,18.8333,-99.1833,25.00,47,100,1.58,MX,1714969946,Ex-Hacienda de Cortes.
92,92,margaret river,-33.9500,115.0667,21.06,50,31,3.63,AU,1714970128,Quality Inn Margaret River
109,109,tsengel,49.4783,100.8894,21.58,21,48,2.80,MN,1714970131,No hotel found
151,151,dingtao,35.0683,115.5625,25.94,35,47,3.25,CN,1714970138,No hotel found
152,152,inda silase,14.1031,38.2829,23.98,15,25,1.21,ET,1714970138,ቃሌማ ሆቴል
160,160,murzuq,25.9155,13.9184,23.88,26,99,5.44,LY,1714970140,فندق باريس - مرزق
189,189,bahia de kino,28.8333,-111.9333,23.29,47,6,2.88,MX,1714970145,Casa Blanca Inn
214,214,saint-pierre,-21.3393,55.4781,23.82,43,0,2.57,RE,1714970149,La Villa Delisle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    size=hotel_df['Humidity'],
    color='City',  # Differentiate between cities with different colors
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],  # This will create the hover tooltip
    tiles='OSM',
    frame_width=700,
    frame_height=500
)



# Display the map
hvplot.show(hotel_map)